# BERT (Bidirectional Encoder Representations from Transformers)

- 참고
    - BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding (Devlin et al., 2019)
    - BERT 논문 정리 (https://tmaxai.github.io/post/BERT/)
    - BERT 논문 리뷰 (https://youtu.be/IwtexRHoWG0)

<img src = "./image/bert_structure.png">

# 1. Introduction

## 1.1. BERT
- BERT는 두 단계로 구성: pre-training과 fine-tuning
    - Pre-training 단계에서 모델은 라벨이 없는 데이터를 기반으로 다양한 pre-training task에 대해 학습
    - Fine-tuning 단계에서는 BERT 모델이 먼저 pre-trained 매개변수로 초기화되며, 모든 매개변수가 downstream tasks의 라벨이 지정된 데이터를 사용하여 fine-tuning 됨
    - 각 downstream task마다 별도의 fine-tuned 모델이 있지만, 모두 같은 pre-trained 매개변수로 초기화됨


- BERT는 레이어의 모든 단계에서 좌우 문맥 모두를 고려하여 양방향(bidirectional)으로 사전 학습된 깊은 표현을 생성하기 위해 설계됨 
- 이를 위해서 BERT는 먼저 대규모의 텍스트 데이터를 사용하여 사전 학습(pretraining)
- 이 사전 학습 단계에서는 양방향 transformer encoder를 사용하여 입력 문장의 모든 토큰의 표현을 생성
- 이 때, BERT는 각 토큰의 왼쪽 문맥과 오른쪽 문맥을 동시에 고려하여 표현을 생성
- 이를 통해 BERT는 양방향(bidirectional) 정보를 사용하여 각 토큰의 표현을 생성하고, 이를 이용해 다양한 자연어 처리 태스크를 수행할 수 있음


- 결과적으로, 사전 학습된 BERT 모델은 추가적인 출력 레이어(output layer) 하나만으로도 특정 구조의 수정 없이 질문 응답(question answering) 및 언어 추론(language inference)과 같은 다양한 작업에 대한 최첨단 모델을 생성하고 fine-tuning 할 수 있음
- 이는 BERT 모델이 대규모의 다양한 데이터로 사전 학습되어서 다양한 자연어 처리 태스크에 대해 일반화(generalization)된 표현을 제공하므로 가능한 것
- 이렇게 생성된 모델은 대규모 데이터셋에서 높은 정확도를 보이며, 적은 양의 task-specific한 학습 데이터로도 높은 성능을 발휘할 수 있음

## 1.2. Others

<img src = "./image/bert_pretraining.png">
- pre-trained language representation을 하위 태스크에 적용하는 데는 두 가지 전략이 있음
- feature-based와 fine-tuning
    - feature-based 접근 방식은 ELMo (Peters et al., 2018a)와 같이 사전 학습된 표현을 추가적인 특징(feature)으로 사용하는 태스크 특정 아키텍처를 사용
    - fine-tuning 접근 방식은 Generative Pre-trained Transformer (OpenAI GPT) (Radford et al., 2018)와 같이 최소한의 태스크 특정 매개변수를 도입하며, 모든 사전 학습된 매개변수를 fine-tuning 함으로써 하위 태스크에 대해 학습
- 이 두 가지 접근 방식은 사전 학습 중에 동일한 object function을 공유
- 이때, 양방향 언어 모델을 사용하여 일반적인 언어 표현을 학습
- 이러한 사전 학습은 문장이나 토큰 수준에서의 다양한 자연어 처리 태스크를 해결하는 데 도움이 되며, 하위 태스크에 따라 적합한 전략을 선택하여 적용할 수 있음

## 1.3. So...
- 논문에서는 현재 기술들이 사전 학습된 표현의 성능을 제한한다고 주장 (특히 fine-tuning 접근 방식) 
- 주요 제한 사항은 표준 언어 모델이 단방향적이기 때문에 사전 학습 중에 사용할 수 있는 아키텍처의 선택이 제한된다는 것
    - 예를 들어, OpenAIGPT에서는 이전 토큰에만 어텐션을 주는 left-to-right 아키텍처를 사용
    - 이러한 제한은 문장 수준 태스크에 대해서는 최적이 아니며, 질문 응답과 같은 토큰 수준의 태스크에 fine-tuning 접근 방식을 적용할 때 매우 유해할 수 있음
    - 이러한 경우 양방향에서의 문맥을 모두 포함하는 것이 매우 중요하기 때문
    
    
- 이 논문에서는, BERT를 제안함으로써 fine-tuning 접근 방식을 개선
- BERT의 Pre-Training
    - **Masked Language Model (MLM)**
        - MLM은 입력에서 일부 토큰을 무작위로 마스킹함
        - 해당 단어의 문맥에만 기반하여 마스킹된 단어의 원래 어휘를 예측하는 것
        - left-to-right 방식의 사전 학습과는 달리, MLM은 표현이 좌우 문맥을 융합할 수 있게 하므로, deep bidirectional함
            - GPT는 앞의 단어를 보고 뒤의 단어를 예측하는 Transformer decoder 구조 사용.  
            - BERT에서는 input 전체와 마스킹된 token을 한번에 Transformer encoder에 넣고 원래 token 값을 예측
    - **Next Sentence Prediction (NSP)**
        - 두 문장을 pre-training시에 같이 넣어줘서 두 문장이 이어지는 문장인지 아닌지 맞추는 것
        - pre-training시에는 50:50 비율로 실제로 이어지는 두 문장과 랜덤하게 추출된 두 문장을 넣어줘서 BERT가 맞추게 시킴
        - 이러한 task는 실제 Natural Language Inference와 같은 task를 수행할 때 도움 됨

# 2. BERT

## 2-1. Model Architecture

- BERT 모델의 아키텍처는 Vaswani et al. (2017)에서 처음 소개되었으며, tensor2tensor 라이브러리에서 발표된 것과 거의 동일
- 이 모델은 멀티레이어 양방향 Transformer encoder로 구성되어 있음
- BERT는 모델의 크기에 따라 base 모델과 large 모델을 제공
    - BERT_base : L=12, H=768, A=12, Total Parameters = 110M
    - BERT_large : L=24, H=1024, A=16, Total Parameters = 340M
    - L : transformer block의 layer 수, H : hidden size, A : self-attention heads 수, feed-forward/filter size = 4H

## 2-2. Input/Output Representations

<img src = "./image/bert_input_representation.png">

- BERT는 다양한 down-stream task를 처리하기 위해, 하나의 token sequence에 single sentence와 pair of sentences(예: 질문, 답변)을 명확하게 나타낼 수 있는 input을 사용
- BERT의 input은 해당하는 Token, Segment, Position Embedding을 합산하여 구성
    - Token은 WordPiece embedding을 사용하고, BERT english의 경우 30000개의 token을 사용
    - Segment는 [SEP]을 기준으로 왼쪽은 E^A, 오른쪽은 E^B로 되어 있음. 첫 번째 시퀀스냐 두 번째 시퀀스냐에 대한 정보를 줌
    - Position은 Transformer와 동일
- 모든 sentence의 첫번째 token은 언제나 [CLS] (special classification token)
    - [CLS] token은 transformer 전체층을 다 거치고 나면 token sequence의 결합된 의미를 가지게 되는데, 
    - 여기에 간단한 classifier를 붙이면 단일 문장, 또는 연속된 문장의 classification을 쉽게 할 수 있게 됨
    - 만약 classification task가 아니라면 이 token은 무시
- 문장을 구분하기 위해 두 가지 방법을 사용
    - 첫째, special token ([SEP])으로 분리
    - 둘째, 문장 A 또는 B에 속하는지를 나타내는 학습된 임베딩을 각 토큰에 추가
    - input embedding을 E로 표시하고, special [CLS] token의 e final hidden vector를 C ∈ R^H로 표시하며, 
    - i번째 입력 토큰의 e final hidden vector를 Ti ∈ R^H로 표시

## 2.3. Pre-training BERT
- 기존의 ELMO나 GPT는 left to right or right to left Language Model을 사용하여 pre-training을 하지만, BERT는 이와 다르게 2가지의 새로운 unsupervised prediction task로 pre-training을 수행

### 2.3.1. Task #1: Masked LM
- 전체 시퀀스의 15%가 랜덤하게 [MASK] token으로 바뀜
- 그림에서 W4가 마스킹되어서 입력으로 들어가고 BERT 모델에서 block들을 거쳐 최종적으로 O4로 출력
- 그 위에 Fully-connected layer + GELU + Norm을 거쳐 실제 토큰인 W'4로 출력되도록 학습시킴
    
<img src = "./image/bert_mlm.png">

- 논문에서는 MLM의 문제점으로, 해당 과정은 pre-training하는 과정에서만 일어나고 fine-tuning 단계에서는 이루어지지 않기 때문에 [MASK] 토큰에 해당하는 미스매치가 발생할 수 있다고 언급
    - 이에 대한 solution으로 실제 fine-tuning 과정에서 i번째 토큰이 마스킹이 될 거라고 선택이 되면, 그 중에 80%만 실제로 마스킹하고 10%는 랜덤한 토큰으로 치환하고, 10%는 마스킹을 하지 않고 그대로 쓰는 방법 제시
    - 예)
        - (80%) my dog is hairy -> my dog is [MASK]
        - (10%) my dog is hairy -> my dog is apple
        - (10%) my dog is hairy -> my dog is hairy
    - 몇 가지 조합으로 실험을 해봤을 때, 이런 비율로 했을 때 교차검증 성능이 잘 나왔기 때문

### 2.3.2. Task #2: Next Sentence Prediction (NSP)
- QA(Question-Answering)와 NLI(Natural Language Inference)는 기본적으로 두 개 이상의 문장들에 대한 관계를 이해해야만 풀 수 있음. 문장 단위의 Language Modeling은 이 관계를 잘 이해할 수 없다고 지적
- 따라서 BERT에서는 두 개 이상의 문장의 관계를 알아야지만 풀 수 있는 NLP Task에 좋은 성능을 나타내기 위해 A Binarized next sentence prediction 모델을 만듦
- monolingual corpus에서 쉽게 생성할 수 있는 binarized next sentence prediction 태스크를 pre-training
    - 문장 A와 B를 선택할 때,
    - 50% 비율로 B는 실제로 A에 이어진 문장 (IsNext)
    - 50%는 실제로 다음 문장이 아닌 것 (NotNext)
    - 마지막 단계에서 Next Sentence인지 아닌지에 대한 Yes or No를 판별하게 됨
- 이 아이디어의 단순함에도 불구하고, QA와 NLI 태스크에 대해 매우 유의한 성능 개선을 보였다고 언급

## 2.4. Pre-training Procedure
- BookCorpus (Zhu et al., 2015) (800M words)와 English Wikipedia (2,500M words)를 사용
- Wikipedia 데이터에서는 text passage만 추출. long contiguous sequence만을 학습시키기 위함
- Hyper-parameter settings
    - Maximum token length: 512
    - Batch size: 256
    - Adam with learning rate of 1e-4, betal = 0.9, beta2 = 0.999
    - L2 weight decay of 0.01
    - Learning rate warmup over the first 10,000 steps, linear decay of the learning rate
    - Dropout probability of 0.1 on all layers
    - GeLU activation fuction rather than standard ReLU
    - BERT_base took 4 days with 16 TPUs and BERT_large took 4 days with 64 TPUs
    - Pre-train the model with sequence length of 128 for 90% of the steps
    - The rest 10% of the steps are trained with sequence length of 512

## 2.5. Fine-tuning BERT

- sequence-level classification tasks에 대해서는 BERT fine-tuning과정이 매우 straightforward
    - input sequence에 대해서 일정한 차원수의 representation 결과를 얻고 싶기 때문에, [CLS] token의 Transformer output값을 사용
    - [CLS] token의 벡터는 H차원을 가짐. C∈RH
    - 여기서 classify하고 싶은 갯수(K)에 따라 classification layer를 붙임. classification layer : W∈RK×H
    - label probabilities는 standard softmax로 계산. P=softmax(CWT)
    - W matrix와 BERT의 모든 파라미터가 같이 fine-tuning 됨
- span-level, token-level prediction tasks의 경우에는, 위의 과정에서 약간 변형시켜 fine-tuning이 진행
- fine-tuning시의 Hyper-Parameter
    - 몇가지를 제외하고는 pre-training때의 hyper parameter와 대부분 동일
    - 다른점은 batch size, learning rate, trainig epochs 수
    - optimal hyperparameter는 task마다 달라지지만, 다음에 제시하는 것을 사용하면 대부분 잘 학습됨
    - Batch size: 16, 32 Learning rage (Adam): 5e-5, 3e-5, 2e-5 Number of epochs : 3, 4

- fine-tuning시의 dataset의 크기가 클수록 hyperparameter에 영향을 덜 받고 잘 training 됨을 관측함
- Fine-tuning은 굉장히 빠르게 학습되며(pre-training에 비해) 이로 인해 최적의 hyperparameter 탐색을 exhaustive search로 찾아내도 무방

## 2.6. Comparison of BERT and OpenAI GPT
- OpenAI GPT와 BERT는 transformer구조를 사용한다는 점에 있어서 공통점을 갖지만, BERT가 훨씬 좋은 성능을 가짐.
- 이 차이는 앞에 설명한 - MLM task, NSP task를 수행하는 것과 별개로 또 더 다른점을 갖기에 생김
    - GPT의 경우 BookCorpus(800M words)만을 pre-training에 사용; BERT는 거기에 + Wikipedia(2500M words) 사용
    - GPT는 [SEP]과 [CLS] token을 fine-tuning시에만 추가하여 학습; BERT는 pre-training시에도 학습 (NSP task가 존재하기 때문에 가능)
    - GPT : 32,000 words/batch for 1M steps ; BERT : 128,000 words/batch ofr 1M steps
    - GPT는 모든 fine-tuning을 수행할 때 learning rate : 5e-5를 사용; BERT는 task-specific하게 조절하여 사용

# 3. Experiments
- 11개의 NLP tasks에 대한 BERT fine-tuning에 대해서 설명
- https://tmaxai.github.io/post/BERT/ 참고

# 4. Ablation Studies

- 이전 챕터에서 중요한 요소라고 설명했던 부분을을 하나씩 제거하며 요소들의 중요함을 파악

## 4.1. Effect of Pre-training Tasks

- BERT_base와 동일한 hyperparameter로 실험을 진행하지만 ablation한 두가지 다른 모델로 실험을 진행
    - No NSP: MLM은 사용하지만, 다음 문장 예측 (NSP)를 없앤 모델
    - LTR & No NSP : MLM 대신 Left-to-Right (LTR) 을 사용하고, NSP도 없앤 모델, 이는 OpenAI GPT모델과 완전히 동일하지만, 더 많은 트레이닝 데이터를 사용
    
<img src = "./image/bert_ablation_result1.png">

- pre-training task를 하나라도 제거하면 성능이 굉장히 떨어짐
- No NSP의 경우에는 NLI계열의 task에서 성능이 많이 하락하게 되는데, 이는 NSP task가 문장간의 논리적인 구조 파악에 중요한 역할을 하고 있음을 알 수 있음
- MLM대신 LTR을 쓰게 되면 성능하락은 더욱더 심해짐. BiLSTM을 더 붙여도, MLM을 쓸 때보다 성능이 하락하는 것으로 보아, MLM task가 더 Deep Bidirectional한 것임을 알 수 있음

## 4.2. Effect of Model Size

- 측정한 데이터셋에서는 모두 모델이 커질수록, 정확도가 상승함

<img src = "./image/bert_ablation_result2.png" width = 600>

- 번역 task나 language modeling과 같은 large-scale task는 모델 사이즈가 클 수록 성능은 계속 상승
- 특히 BERT의 경우에는, downstream task를 수행하는 dataset의 크기가 작아도, pre-training덕분에, model의 크기가 클 수록 정확도는 상승함

## 4.3. Feature-based Approach with BERT

- 지금까지 BERT는 pre-training을 진행 한 후, downstream task를 학습 할 때, 간단한 classifier를 부착해서 모든 layer를 다시 학습시키는 fine-tuning 방법을 사용하는 것으로 설명
- 하지만 BERT를 ELMO와 같이 feature based approach로도 사용을 할 수 있음
- Feature-based Approach는 몇가지 이점이 존재
    - Transformer encoder는 모든 NLP task를 represent하지는 못하므로, 특정 NLP task를 수행할 수 있는 Network를 부착하여 쓸 수 있음
    - Computational benefit을 얻을 수 있음
- 해당 section에서는 BERT를 ELMO와 같이 마지막 레이어에 Bi-LSTM을 부착시켜, 해당 레이어만 학습 시키는 방법론을 사용

<img src = "./image/bert_ablation_result4.png" width=600>

- 표에서 볼 수 있듯이, Concat Last Four Hiddem값을 사용하면, Finetune All과 단지 0.3 F1 score차이밖에 나지 않음
- 이를 통해, BERT는 Feature-based Approach에서도 효과적이라고 할 수 있음

- https://mccormickml.com/2019/07/22/BERT-fine-tuning/